In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../..")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from pathlib import Path
import pickle
import timeit
#
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, models
from dotted_dict import DottedDict
from tqdm import tqdm
import pprint
#
from csprites.datasets import ClassificationDataset
import utils
from backbone import get_backbone
from optimizer import get_optimizer
from sklearn.linear_model import LogisticRegression
from plot_utils import *
from eval_utils import load_data_tensors

# Settings

In [ ]:
# linprob config
linprob_config = {
    'p_eval': 'eval',
    'p_eval_results': 'results_lr.pkl',
    'p_config': "linprob_config_lr.pkl",
    'p_results': 'results_lr.pkl',
    'p_acc_mat': 'accuracy_lr.png',
}
linprob_config = DottedDict(linprob_config)

In [ ]:
p_experiments_base = Path("/mnt/experiments/csprites/single_csprites_64x64_n7_c24_a32_p13_s3_bg_inf_random_function_77000/")
p_experiments = [
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[angle]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[color]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[px]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[py]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[scale]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[shape]',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_32]_2021-09-14_21-47-07',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_64]_2021-09-14_21-19-32',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_128]_2021-09-14_20-52-16',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_256]_2021-09-14_20-21-16',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_32]_2021-09-14_19-50-04',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_64]_2021-09-14_19-17-32',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_128]_2021-09-14_18-44-50',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_256]_2021-09-14_18-15-36',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_32]_2021-09-14_17-38-13',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_64]_2021-09-14_17-01-16',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_128]_2021-09-14_16-22-39',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_256]_2021-09-14_15-44-33',

]
for p in p_experiments:
    if not p.exists():
        print("NOT EXISTENT: \n", p)

In [ ]:
step = 1
max_steps = len(p_experiments) * 6
print_str = "[{:>3}/{:>3}] {:<10} {}"
for p_experiment in p_experiments:
    experiment_name = p_experiment.name
    
    # experiment config
    p_experiment_config = p_experiment / "config.pkl"
    with open(p_experiment_config, "rb") as file:
        experiment_config = pickle.load(file)
    # dataset config
    p_ds_config = Path(experiment_config.p_data) / "config.pkl"
    with open(p_ds_config, "rb") as file:
        ds_config = pickle.load(file)

    results = {}
    for target_variable in ds_config["classes"]:
        print(print_str.format(step, max_steps, target_variable, experiment_name))
        target_idx = [idx for idx, target in enumerate(ds_config["classes"]) if target == target_variable][0]
        n_classes = ds_config["n_classes"][target_variable]
        #
        R_train, Y_train, R_valid, Y_valid = load_data_tensors(
                target_idx,
                p_experiment / experiment_config["p_R_train"],
                p_experiment / experiment_config["p_Y_train"],
                p_experiment / experiment_config["p_R_valid"],
                p_experiment / experiment_config["p_Y_valid"])
        #
        clf = LogisticRegression(random_state=0).fit(R_train, Y_train)
        score = clf.score(R_valid, Y_valid)
        print("   LINPROB: {:.3f}".format(score))
        results[target_variable] = score
        step += 1
    p_results = p_experiment / linprob_config["p_results"]
    p_linprob_config = p_experiment / linprob_config["p_config"]
    #
    plt.bar(range(R_train.shape[1]),R_train.mean(axis=0), width=1)
    plt.title("TRAIN REPRESENTAIONS")
    plt.savefig(p_experiment / "R_train_feature_mean.png")
    plt.show()
    plt.bar(range(R_valid.shape[1]),R_train.mean(axis=0), width=1)
    plt.title("VALID REPRESENTAIONS")
    plt.savefig(p_experiment / "R_valid_feature_mean.png")
    plt.show()
    #
    with open(p_results, "wb") as file:
        pickle.dump(results, file)
    with open(p_linprob_config, "wb") as file:
        pickle.dump(linprob_config, file)

# Load all results 

In [ ]:
p_experiments_base = Path("/mnt/experiments/csprites/single_csprites_64x64_n7_c24_a32_p13_s3_bg_inf_random_function_77000/")
p_experiments = [
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[shape]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[scale]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[color]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[angle]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[py]',
    p_experiments_base / 'SUP_[FCN16i223o64_d64]_target_[px]',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_32]_2021-09-14_21-47-07',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_64]_2021-09-14_21-19-32',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_128]_2021-09-14_20-52-16',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN8i223o32_d_256]_2021-09-14_20-21-16',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_32]_2021-09-14_19-50-04',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_64]_2021-09-14_19-17-32',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_128]_2021-09-14_18-44-50',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN16i223o64_d_256]_2021-09-14_18-15-36',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_32]_2021-09-14_17-38-13',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_64]_2021-09-14_17-01-16',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_128]_2021-09-14_16-22-39',
    p_experiments_base / 'BarlowTwins' / 'BTwins_[FCN32i223o128_d_256]_2021-09-14_15-44-33',

]
for p in p_experiments:
    if not p.exists():
        print("NOT EXISTENT: \n", p)

In [ ]:
p_eval =  p_experiments_base / linprob_config["p_eval"]
p_eval.mkdir(exist_ok=True)
p_results = p_eval / linprob_config["p_results"]
p_config = p_eval / linprob_config["p_config"]
p_acc_map = p_eval / linprob_config["p_acc_mat"]

In [ ]:
experiment_results = {}
for p_experiment in p_experiments:
    experiment_name = p_experiment.name
    print(experiment_name)
    #
    p_results = p_experiment / linprob_config["p_results"]
    #
    with open(p_results, "rb") as file:
        results = pickle.load(file)
    experiment_results[experiment_name] = results

In [ ]:
acc_results = []
for experiment_name in experiment_results.keys():
    results = experiment_results[experiment_name]
        # targets
    targets = list(results.keys())
    accs = [results[t] for t in targets]
    acc_results.append(accs)
acc_matrix = np.array(acc_results)
path_experiment_names = [n for n in experiment_results.keys()]
#
experiment_names = []
for name in path_experiment_names:
    if "_2021-09" in name:
        experiment_names.append(name[:-21])
    else:
        experiment_names.append(name)

In [ ]:
plot_mat(acc_matrix, experiment_names, targets, scale_factor=1, p_file=p_acc_map)

In [ ]:
for acc, name in zip(acc_matrix.sum(axis=1), experiment_names):
    print("{:>40}: {:.3f}".format(name, acc))

In [ ]:
for p in p_experiments:
    R_train = np.load(p / "R_train.npy")
    print("{:<40}: {:.3f} {:.3f} {:.3f}".format(p.name, R_train.min(), R_train.mean(),R_train.max()))

In [ ]:
for p in p_experiments:
    R_train = np.load(p / "R_train.npy")
    print("{:<40}: {:.3f} {:.3f} {:.3f}".format(p.name, R_train.min(), R_train.mean(),R_train.max()))
    cov = np.cov(R_train.T)
    plt.imshow(cov)
    plt.colorbar()
    plt.show()